# Bibliotecas Usadas

In [1]:
import numpy as np
import bisect 
import time
import random
import copy

import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import display
import threading

# O Jogo

Abaixo, está a classe elaborada para simular o jogo slide number, necessário para fazer as possibilidades de partida 

In [2]:
class Jogo:
    
    """
    Para o construtor, é necessário informar a dimensão, do tabuleiro quadrado, e uma jogada anterior
    que, por default, não existe. O construtor faz um tabuleiro gabarito
    """
    def __init__(self, dimensao, jogada_anterior=None):
        
        tabuleiro = []
        peso = {}
        coordinate = {}
        k=1
        
        #loop tradicional para fazer uma matriz
        for i in range(0,dimensao):
            tabuleiro.append([])
            for j in range(0, dimensao):
                tabuleiro[i].append(k)
                peso[k] = 0
                coordinate[k] = (i, j)
                k+=1
        
        #Espaço vazio do jogo                        
        tabuleiro[-1][-1] = None
        peso.pop(dimensao**2)
        coordinate.pop(dimensao**2)
        
        peso[None] = 0
        coordinate[None] = (dimensao-1, dimensao-1)
        
        #posição do vazio
        self.vazio = (dimensao-1, dimensao-1)
        
        #tabuleiro
        self.tabuleiro = tabuleiro
        
        #dimensão do tabuleiro
        self.dimensao = dimensao
        
        self.peso = peso
        self.coordinate = coordinate
        
        #jogada anterior que gerou o tabuleiro atual
        self.jogada_anterior = jogada_anterior
    
    """
    Função para copiar um tabuleiro e mudar sua referencia
    """
    def copy(self):
        return copy.deepcopy(self)
    
    def mudaPeso(self, valor, pos, gabarito):
        pos_final = gabarito.coordinate[valor]
        valor_peso = abs(pos_final[0] - pos[0]) + abs(pos_final[1] - pos[1])
        self.peso[valor] = valor_peso
        
    """
    Função que simula uma jogada legal. Retorna outro tabuleiro com a jogada
    """
    def jogada(self, pos, gabarito):
        
        #presenvando o tabuleiro atual
        novo = self.copy()
        
        #pegando os vizinhos do vazio
        vizinhos = novo.getVizinhos()
        
        #se a jogada for um vizinho, a pos é valida!
        if pos in vizinhos:
            
            valor = novo.tabuleiro[pos[0]][pos[1]]
            
            novo.tabuleiro[self.vazio[0]][self.vazio[1]] = valor
            
            self.mudaPeso(valor, self.vazio, gabarito)  
            
            novo.tabuleiro[pos[0]][pos[1]] = None
            
            self.mudaPeso(None, pos, gabarito)
            
            novo.vazio = pos
            
            novo.jogada_anterior = pos
            
            novo.peso = self.peso
            
        return novo
    
    """
    Função que retorna as posições de jogadas valídas
    """
    def getVizinhos(self):
        
        #vendo os vizinhos do vazio
        pos = self.vazio
        possibilidades = [
            (pos[0]-1, pos[1]),
            (pos[0]+1, pos[1]),
            (pos[0], pos[1]+1),
            (pos[0], pos[1]-1)
        ]
        
        #tirando os que são ilegais
        possibilidades = filter(lambda x: (x[0] >= 0 and x[0] < self.dimensao) and 
               (x[1] >= 0 and x[1] < self.dimensao), possibilidades)
        
        return list(possibilidades)
    
    """
    Função que cria um tabuleiro randomico partindo de um gabarito
    """    
    def embaralha(self, vezes, gabarito):
        
        #presenvando o tabuleiro atual
        proximo = self.copy()
        
        #fazundo multiplas jogadas aleatorias
        for i in range(0,vezes):
            x = proximo.getVizinhos()
            
            pos = random.choice(x)
            
            proximo = proximo.jogada(pos, gabarito)
            
        return proximo
    
    """
    Função para verificar igualdade entre tabuleiros
    """
    def __eq__(self, outro):
        return (self.vazio == outro.vazio and self.tabuleiro == outro.tabuleiro)
    
    """
    Sobreescrita do print usando numpy
    """
    def __str__(self):
        return str(np.array(self.tabuleiro))
    

# O nó da árvore

Para montar a arvore, é necessário utilizar uma estrutura de nós com n filhos e 1 pai

In [3]:
class No:
    
    """
    Para construir o nó, é necessário passar seu conteúdo e seu pai que, por default, é None
    """
    def __init__(self, conteudo,pai = None):
        
        #pai do nó
        self.pai = pai
        
        #conteudo do nó
        self.conteudo = conteudo
        
        #lista de filhos
        self.filhos = list()
        
        self.pesoTotal = self.calculaPeso()
    
    def calculaPeso(self):
        # modo correto
        # peso_pai = 0 if not self.pai else self.pai.pesoTotal
        # modo errado
        peso_pai = 0
        return sum(self.conteudo.peso.values()) + peso_pai
    
    """
    Funçaõ para adicionar filhos ao nó, deve-se passar apenas o conteudo do filho
    """
    def add(self, conteudo):    
        filho = No(conteudo, pai = self)
        self.filhos.append(filho)
        
    """
    dois nós são iguais se se o conteúdo for o mesmo
    """
    def __eq__(self, outro):
        
        return (self.conteudo == outro.conteudo)
    
    def __lt__(self, other):
        return self.pesoTotal < other.pesoTotal

# Funções auxiliares

Para fazer o busca em largura de forma dinamica, foi necessário elaborar funções de verificação e manuntenção de arvore

In [4]:
"""
Função para verificar se, dado um nó, ele ja apareceu em sua linhagem
"""
def Correspondencia(no):
    if not no.pai:
        return False
    
    proximo = no.pai
    
    while proximo:
        
        if proximo == no:
            return True
        else:
            proximo = proximo.pai
    return False

In [5]:
"""
Função para, dado um nó, gerar seus filhos de jogadas lícitas
"""
def CriarFilhos(no, fim):
    t = no.conteudo
    
    pos = t.getVizinhos()
    
    filhos = []

    for coord in pos:
        
        j = t.jogada(coord, fim.conteudo)
        
        if not Correspondencia(No(j,pai = no)): filhos.append(No(j,pai = no))
    
    return filhos

# A Busca Em Largura Dinâmica

Finalmente, o algoritmo de busca. Ele gera uma arvore de possibilidades até achar uma resposta

In [6]:

"""
Algoritmo para achar o caminho de jogadas do jogo passado como parametro
"""
def BuscaEmLargura(problema, solucao):
    count = 0
    #se o seu problema é igual a solução...
    if problema == solucao:
        return problema
    
    #Gerarndo as primeiras possibilidades
    proximos = CriarFilhos(problema, solucao)
    
    #enquanto tiver possibilidades
    while len(proximos) != 0:
        count += 1
        filho = proximos.pop(0)
        
        #achei a solucao!
        if filho == solucao:
            print("Solved with ", count, "iterations")
            
            return filho
        
        for x in CriarFilhos(filho, solucao): proximos.append(x)
        
    print("Solved with", count, "iterations")
        
    return None  


def AEstrela(problema, solucao):
    count = 0
    #se o seu problema é igual a solução...
    if problema == solucao:
        return problema
    
    #Gerarndo as primeiras possibilidades
    proximos = CriarFilhos(problema, solucao)
    
    proximos.sort(key=lambda x: x.pesoTotal)
    
    #enquanto tiver possibilidades
    while len(proximos) != 0:
        count += 1
        filho = proximos.pop(0)
        
        #achei a solucao!
        if filho == solucao:
            print("Solved with ", count, "iterations")
            
            return filho
        
        for x in CriarFilhos(filho, solucao): bisect.insort(proximos, x)
        
        proximos.sort(key=lambda x: x.pesoTotal)
        
    print("Solved with", count, "iterations")
        
    return None    

In [7]:
"""
Algoritmo para achar o conjunto de jogadas que originaram o nó
"""
def coletarJogadas(no):
    jogadas = []
    proximo = no
    while proximo.pai:
        jogadas.append(proximo.conteudo.jogada_anterior)
        proximo = proximo.pai
    return jogadas[::-1]        

# INTERFACE

In [32]:
class Interface:
    def __init__(self, dimensao):
        self.dimensao = dimensao
        self.gabarito = Jogo(dimensao)
        self.problema = Jogo(dimensao)
        
        columns = "repeat(" + str(dimensao) + ", 54px)"
        self.box = widgets.GridBox(layout=widgets.Layout(grid_template_columns=columns))
        
        self.tabuleiro_dic = {}
        
        self.output = widgets.Output()
        
        buttonWidth = str(54*self.dimensao) + "px"
        
        self.resolverA =  widgets.Button(
                description="A*",
                layout=widgets.Layout(width=buttonWidth, height='50px')
            )
        self.resolverA.on_click(self.getSolucaoA)
        self.resolverA.style.button_color = 'lightblue'

        self.resolverBusca =  widgets.Button(
                        description="BUSCA EM LARGURA",
                        layout=widgets.Layout(width=buttonWidth, height='50px')
                    )
        self.resolverBusca.on_click(self.getSolucaoBusca)
        self.resolverBusca.style.button_color = 'lightgreen'
        
        self.generateGame()
        
    def movimenta(self, valor):
        i, j = self.tabuleiro_dic[valor]
        possiveis = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

        nulo = self.tabuleiro_dic[""]

        if valor != "" and nulo in possiveis:

            novo_nulo = self.tabuleiro_dic[valor]


            self.problema = self.problema.jogada(novo_nulo, gabarito)

            self.tabuleiro_dic[valor] = nulo
            self.tabuleiro_dic[""] = novo_nulo

            return True

        return False
    
    def troca(b):
        new = []

        for botao in self.box.children:
            if botao.description == "":
                new.append(b.description)

            elif botao == b:
                vazio = widgets.Button(
                            description="",
                            layout=widgets.Layout(width='50px', height='50px')
                        )
                vazio.on_click(self.on_button_clicked)

                vazio.style.button_color = 'white'
                new.append(vazio)

            else:
                new.append(botao)

        self.box.children = new
    
    def on_button_clicked(self, b):
        with self.output:
            self.output.clear_output()
            print(type(b))
            if self.movimenta(b.description):
                troca(b)
    
    def generateGame(self):
        botoes = []
        count = 1
        
        for i in range(self.dimensao):
            for j in range(self.dimensao):
                button = widgets.Output()
                if count != self.dimensao**2:
                    button = widgets.Button(
                        description=str(count),
                        layout=widgets.Layout(width='50px', height='50px')
                    )
                    button.on_click(self.on_button_clicked)

                    button.style.button_color = 'pink'

                    self.tabuleiro_dic[str(count)] = (i, j)

                else:
                    button = widgets.Button(
                        description="",
                        layout=widgets.Layout(width='50px', height='50px')
                    )

                    button.on_click(self.on_button_clicked)

                    button.style.button_color = 'white'

                    self.tabuleiro_dic[""] = (i, j)

                botoes.append(button)
                count += 1
                
        self.box.children = botoes
        
    def resolve(self, solucao):
        for tupla in solucao:
            botao = None
            valor = ""

            for key in tabuleiro_dic:
                if tabuleiro_dic[key] == tupla:
                    valor = key

            for bot in self.box.children:
                if bot.description == valor:
                    botao = bot
                    
            if movimenta(valor):
                t = threading.Thread(target=troca,args=(botao,))
                t.start()
                time.sleep(0.4)
                
        self.gabarito = Jogo(3)
        self.problema = Jogo(3)

    def getSolucaoA(b):
        inicio = No(self.problema)
        fim = No(self.gabarito)
        resultado = AEstrela(inicio, fim)
        jogadas = coletarJogadas(resultado)

        resolve(jogadas)

    def getSolucaoBusca(b):
        inicio = No(self.problema)
        fim = No(self.gabarito)
        resultado = BuscaEmLargura(inicio, fim)
        jogadas = coletarJogadas(resultado)

        resolve(jogadas)


In [33]:
inter = Interface(4)

display(inter.box, inter.resolverA, inter.resolverBusca, inter.output)

GridBox(children=(Button(description='1', layout=Layout(height='50px', width='50px'), style=ButtonStyle(button…

Button(description='A*', layout=Layout(height='50px', width='216px'), style=ButtonStyle(button_color='lightblu…

Button(description='BUSCA EM LARGURA', layout=Layout(height='50px', width='216px'), style=ButtonStyle(button_c…

Output()

In [24]:
inter.tabuleiro_dic


{'1': (0, 0),
 '2': (0, 1),
 '3': (0, 2),
 '4': (0, 3),
 '5': (1, 0),
 '6': (1, 1),
 '7': (1, 2),
 '8': (1, 3),
 '9': (2, 0),
 '10': (2, 1),
 '11': (2, 2),
 '12': (2, 3),
 '13': (3, 0),
 '14': (3, 1),
 '15': (3, 2),
 '': (3, 3)}

In [21]:
global box, tabuleiro_dic, resultado, inicio, fim, gabarito, problema

gabarito = Jogo(3)
problema = Jogo(3)

box = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(3, 54px)"))

tabuleiro_dic = {}

def movimenta(valor):
    i, j = tabuleiro_dic[valor]
    possiveis = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
    
    nulo = tabuleiro_dic[""]
    
    if valor != "" and nulo in possiveis:
        
        novo_nulo = tabuleiro_dic[valor]

        global problema
        problema = problema.jogada(novo_nulo, gabarito)
        
        tabuleiro_dic[valor] = nulo
        tabuleiro_dic[""] = novo_nulo

        return True
    
    return False
        
def troca(b):
    new = []
    
    for botao in box.children:
        if botao.description == "":
            new.append(b)

        elif botao == b:
            vazio = widgets.Button(
                        description="",
                        layout=widgets.Layout(width='50px', height='50px')
                    )
            vazio.on_click(on_button_clicked)

            vazio.style.button_color = 'white'
            new.append(vazio)

        else:
            new.append(botao)

    box.children = new

def on_button_clicked(b):
    with output:
        output.clear_output()
        
        if movimenta(b.description):
            troca(b)
        

def generateGame():
    
    botoes = []
    count = 1
    for i in range(3):
        for j in range(3):
            button = widgets.Output()
            if count != 9:
                button = widgets.Button(
                    description=str(count),
                    layout=widgets.Layout(width='50px', height='50px')
                )
                button.on_click(on_button_clicked)
                
                button.style.button_color = 'pink'
                
                tabuleiro_dic[str(count)] = (i, j)
            
            else:
                button = widgets.Button(
                    description="",
                    layout=widgets.Layout(width='50px', height='50px')
                )
            
                button.on_click(on_button_clicked)
                
                button.style.button_color = 'white'
                
                tabuleiro_dic[""] = (i, j)
            
            botoes.append(button)
            count += 1
    box.children = botoes
    return box


def resolve(solucao):
    for tupla in solucao:
        botao = None
        valor = ""
        
        for key in tabuleiro_dic:
            if tabuleiro_dic[key] == tupla:
                valor = key
        
        for bot in box.children:
            if bot.description == valor:
                botao = bot
        if movimenta(valor):
            t = threading.Thread(target=troca,args=(botao,))
            t.start()
            time.sleep(0.4)
    
    global problema, gabarito
    
    gabarito = Jogo(3)
    problema = Jogo(3)
        
def getSolucaoA(b):
    global resultado, inicio, fim, problema, gabarito
    inicio = No(problema)
    fim = No(gabarito)
    resultado = AEstrela(inicio, fim)
    jogadas = coletarJogadas(resultado)
    
    resolve(jogadas)
    
def getSolucaoBusca(b):
    global resultado, inicio, fim, problema, gabarito
    inicio = No(problema)
    fim = No(gabarito)
    resultado = BuscaEmLargura(inicio, fim)
    jogadas = coletarJogadas(resultado)
    
    resolve(jogadas)


In [22]:
output = widgets.Output()
jogo = generateGame()
resolverA =  widgets.Button(
                description="A*",
                layout=widgets.Layout(width='162px', height='50px')
            )
resolverA.on_click(getSolucaoA)
resolverA.style.button_color = 'lightblue'

resolverBusca =  widgets.Button(
                description="BUSCA EM LARGURA",
                layout=widgets.Layout(width='162px', height='50px')
            )
resolverBusca.on_click(getSolucaoBusca)
resolverBusca.style.button_color = 'lightgreen'

display(output)
display(jogo, resolverA, resolverBusca)

Output()

GridBox(children=(Button(description='1', layout=Layout(height='50px', width='50px'), style=ButtonStyle(button…

Button(description='A*', layout=Layout(height='50px', width='162px'), style=ButtonStyle(button_color='lightblu…

Button(description='BUSCA EM LARGURA', layout=Layout(height='50px', width='162px'), style=ButtonStyle(button_c…

Solved with  1516 iterations
Solved with  122 iterations
